# Ручка 3. Summary

Делает краткое саммари всего конспекта (на вход идет весь файл в обсидиане, можно добавить передачу всей папки)

In [64]:
from langchain.chat_models.gigachat import GigaChat
from dotenv import load_dotenv
import os

load_dotenv()

GIGACHAT_CREDENTIALS = os.environ.get("GIGACHAT_CREDENTIALS")

llm = GigaChat(
    credentials=GIGACHAT_CREDENTIALS,
    model="GigaChat-Pro",
    verify_ssl_certs=True,
    scope="GIGACHAT_API_CORP",
)

In [70]:
# Загружаем конспект
from langchain_community.document_loaders.text import TextLoader

notes_path = "./notes/small_note.md"

loader = TextLoader(notes_path, "utf-8")
docs = loader.load()

print(docs[0].page_content)

# Используем Stuff Documents Chain + Summarization Checker

In [74]:
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMSummarizationCheckerChain

def stuff_summary(docs, checker: bool=False, max_checks:int=2):
     prompt_template = """Выдели клюбючевые моменты в тексте.
     Оформи ответ в виде списка.
     Текст: "{context}"
     Ключевые моменты:"""
     prompt = PromptTemplate.from_template(prompt_template)
     stuff_chain = create_stuff_documents_chain(llm, prompt)
     res = stuff_chain.invoke({"context": docs})
     if not checker:
          return res
     else:
          checker_chain = LLMSummarizationCheckerChain.from_llm(llm, max_checks=max_checks)
          return checker_chain.run(res)
          

print(stuff_summary(docs, True, 2))

- Влияние параметров пара на показатели комбинированной установки с простым паровым контуром
- Максимальная мощность паровой турбины достигается при определенных параметрах пара
- Мощность паровой турбины зависит от расхода пара, располагаемого перепада энтальпий на турбину и КПД паровой турбины
- Температура пара перед паровой турбиной обычно принимается максимально возможной для уменьшения потерь от влажности и повышения КПД турбины
- Увеличение давления пара приводит к снижению расхода генерируемого пара и росту температуры уходящих газов
- С ростом давления располагаемый перепад энтальпий на паровую турбину увеличивается, что может привести к снижению КПД проточной части
- Зависимость КПД парогазовой установки от начального давления достаточно слабая, но имеется оптимальное значение, которое увеличивается с ростом температуры газа за газовой турбиной
- Температура уходящих газов остается выше минимально допустимой, и ее можно более полно использовать, введя в схему контур низкого д

# Используем Map-Reduce

In [75]:
from langchain_core.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter


# Map
map_template = """Ниже представлен список документов
{docs}
На основе этих документов определи, пожалуйста, основные темы 
Полезный ответ:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

reduce_template = """Ниже представлен набор кратких изложений:
{docs}
Возьми их и сведи в окончательное, объединенное краткое изложение основных тем. 
Полезный ответ:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=8000,
)

# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=2000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

map_reduce_chain.run(split_docs)

'Основные темы представленных документов включают влияние параметров пара на показатели комбинированной установки с простым паровым контуром, зависимость мощности паровой турбины от расхода пара и КПД паровой турбины, а также влияние давления пара на относительный расход пара и температуру уходящих газов. Кроме того, рассматривается влияние давления на располагаемый перепад энтальпий паровой турбины, зависимость КПД парогазовой установки от начального давления, влияние температуры газа за газовой турбиной на эффективность оборудования, возможность использования теплоты уходящих газов, а также влияние минимального температурного напора на расход пара и температуру уходящих газов.'